# COVID-19 knowledge graph

## Abstract

Relations between chemicals, genes and disease offer insights into the mechanisms behind higher order biochemical phenomena, such as drug-drug interactions, drug response and gene-disease associations.
This work focused on extracting relations between chemicals, genes and disease from research articles to build a biomedical knowledge graph related to COVID-19. We used entities collected by [BioSNAP](http://snap.stanford.edu/biodata/index.html) to identify instances of chemical, gene and disease names in abstracts.

To understand what relationship types are possible and map unstructured natural language descriptions onto these structured classes, we used labeled sentences in [Percha B 2018](https://academic.oup.com/bioinformatics/article/34/15/2614/4911883) to classify relations bewteen entities.
The source code for classification model can be seen [here](https://github.com/jxzly/Biomedical-Relation-Classification).

In [1]:
!pip install pyecharts

     |████████████████████████████████| 128 kB 2.8 MB/s 
     |████████████████████████████████| 83 kB 884 kB/s 
  Created wheel for simplejson: filename=simplejson-3.17.0-cp36-cp36m-linux_x86_64.whl size=121382 sha256=62d8410181bfee3c05f0e1722293be42ae6d194b1ed997f12573e984ca131f80
  Stored in directory: /root/.cache/pip/wheels/c2/95/a2/477b46cfe980061fddc4cb76ed4657eceec35d2b2090355e41
Successfully built simplejson


In [2]:
import numpy as np 
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from pyecharts import options as opts
from pyecharts.charts import Graph
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset,DataLoader
from keras.preprocessing import sequence
from transformers import BertTokenizer, BertForSequenceClassification

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

Using TensorFlow backend.


# Load marked sentences

These sentences containing more than two biomedical entities were extracted from abstracts. We combined the pairwise entities in each sentence to build relations.


In [3]:
marked_sentence_df = pd.read_csv('/kaggle/input/covid19-knowledge-graph/marked_sentence.csv')

In [4]:
marked_sentence_df.sample(5)

,start_entity,end_entity,start_entity_type,end_entity_type,marked_sentence
18508,E2,"Dermatitis, Atopic",Gene,Disease,"Delsing, Bee Wah Lee, Bee Wah Lee, Si Hui Goh,..."
8485,alanine,aspartate aminotransferase,Gene,Gene,The results of laboratory examination showed t...
18823,Anaphylaxis,Dermatitis,Disease,Disease,"Ehlayel, Abdul Bari Bener A607 Strong Associat..."
308,IFNA1,IL4,Gene,Gene,Despite these positive effects on proliferatio...
10388,FKBP1B,PPIH,Gene,Gene,"In this study, we used a systems biology appro..."


In [5]:
def Build_graph(df,relation=False,repulsion=40,title='COVID-19 knowledge graph',labelShow=False):
    entity_type_dic = dict(df.drop_duplicates(['start_entity']).set_index(['start_entity'])['start_entity_type'])
    entity_type_dic.update(dict(df.drop_duplicates(['end_entity']).set_index(['end_entity'])['end_entity_type']))
    color = {'Disease':'#FF7F50','Gene':'#48D1CC','Chemical':'#B3EE3A'}
    cate =  {'Disease':0,'Gene':1,'Chemical':2}
    categories = [{'name':'Disease','itemStyle': {'normal': {'color': color['Disease']}}},{'name':'Gene','itemStyle': {'normal': {'color': color['Gene']}}},{'name':'Chemical','itemStyle': {'normal': {'color': color['Chemical']}}}]
    nodes = []
    for entity in list(set(df['start_entity'])|set(df['end_entity'])):
        nodes.append({'name': entity, 'symbolSize': max(10,np.log1p(df.loc[(df['start_entity']==entity)|(df['end_entity']==entity)].shape[0])*10//1),
                     'category':cate[entity_type_dic[entity]]})
    links = []
    for i in df.index:
        if not relation:
            links.append({'source': df.loc[i,'start_entity'], 'target': df.loc[i,'end_entity']})
        else:
            links.append({'source': df.loc[i,'start_entity'], 'target': df.loc[i,'end_entity'], 'value':df.loc[i,'pred']})
    g = (
        Graph()
        .add('', nodes, links,categories, repulsion=repulsion,label_opts=opts.LabelOpts(is_show=labelShow))
        .set_global_opts(title_opts=opts.TitleOpts(title=title),legend_opts=opts.LegendOpts(orient='vertical', pos_left='2%', pos_top='40%',legend_icon='circle'))
        .render_notebook()
        )
    return g

In [6]:
g = Build_graph(marked_sentence_df.sample(100),title='subsample of topology graph')
g

# Classify relations

[Percha B 2018](https://academic.oup.com/bioinformatics/article/34/15/2614/4911883) revealed 10 broad themes for chemical-gene relations, 7 for chemical-disease, 10 for gene-disease and 9 for gene–gene in Medline abstracts. We used these labeled sentences to train a model to classify sentences in this work. As we cannot confirm order of entities, we predicted two possibilities(init_pred and reverse_pred) and retrieved the higher one. The details can be seen [here](https://github.com/jxzly/Biomedical-Relation-Classification)


In [7]:
class Args:
    task_type = 'chemical-disease'
    max_seq_len = 64
    bs = 64

class Conf:
    # some information can be found in:
    # Percha B, Altman R B. A global network of biomedical relationships derived from text[J]. Bioinformatics, 2018, 34(15): 2614-2624.
    relation_type = {'chemical-disease':['T', 'C', 'Sa', 'Pr', 'Pa', 'J'],
                     'disease-chemical':['Mp'],
                     'chemical-gene':['A+', 'A-', 'B', 'E+', 'E-', 'E', 'N'],
                     'gene-chemical':['O', 'K', 'Z'],
                     'gene-disease':['U', 'Ud', 'D', 'J', 'Te', 'Y', 'G'],
                     'disease-gene':['Md', 'X', 'L'],
                     'gene-gene':['B', 'W', 'V+', 'E+', 'E', 'I', 'H', 'Rg', 'Q'],
                     }

args = Args()
conf = Conf()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [8]:
# load pretrained Bert model
def Bert_model(taskType,bertPath):
    label_df = pd.read_csv('/kaggle/input/covid19-knowledge-graph/%s_label.csv'%taskType)
    tokenizer = BertTokenizer.from_pretrained(bertPath,do_lower_case=False)
    model = BertForSequenceClassification.from_pretrained(bertPath, num_labels=label_df['label'].nunique())
    return label_df,tokenizer,model

# bulid data loader
def Data_loader(x,y=None,bs=128,shuffle=False,numWorkers=0):
    if y is not None:
        data = TensorDataset(x,y)
    else:
        data = TensorDataset(x)
    data_loader = DataLoader(dataset=data,batch_size=bs,shuffle=shuffle,num_workers=numWorkers)
    return data_loader

def Prepare_predict_data(tokenizer,bs):
    marked_sentences = marked_sentence_df.loc[(marked_sentence_df['start_entity_type'].apply(lambda x:x.lower())==args.task_type.split('-')[0])&\
                                              (marked_sentence_df['end_entity_type'].apply(lambda x:x.lower())==args.task_type.split('-')[1]),'marked_sentence']
    # convert tokens to ids
    ids = marked_sentences.apply(lambda x:tokenizer.convert_tokens_to_ids(tokenizer.tokenize(x))).tolist()
    # padding ids
    ids = sequence.pad_sequences(ids,args.max_seq_len, truncating='post', padding='post')
    # we cannot confirm order of entities, so predict two possibilities
    reverse_marked_sentences = marked_sentence_df.loc[(marked_sentence_df['start_entity_type'].apply(lambda x:x.lower())==args.task_type.split('-')[0])&\
                                              (marked_sentence_df['end_entity_type'].apply(lambda x:x.lower())==args.task_type.split('-')[1]),'marked_sentence']\
                                              .apply(lambda x:x.replace('start_entity','init_start_entity').replace('end_entity','start_entity').replace('init_start_entity','end_entity'))
    reverse_ids = reverse_marked_sentences.apply(lambda x:tokenizer.convert_tokens_to_ids(tokenizer.tokenize(x))).tolist()
    reverse_ids = sequence.pad_sequences(reverse_ids,args.max_seq_len, truncating='post', padding='post')
    predict_data_loader = Data_loader(torch.LongTensor(ids),torch.LongTensor(reverse_ids),bs=bs)
    return marked_sentences.values,predict_data_loader

def Predict():
    reverse_task_type = args.task_type.split('-')[1] + '-' + args.task_type.split('-')[0]
    def Filter(x):
        if x['init_pred'] in conf.relation_type[args.task_type]:
            if x['reverse_pred'] not in conf.relation_type[reverse_task_type]:
                # init_pred is a correct relation but reverse_pred not
                return 'init_pred'
            else:
                # init_pred and reverse_pred both are correct relations
                if x['init_pred_prob'] >= x['reverse_pred_prob']:
                    # init_pred_prob greater than or equal to reverse_pred_prob
                    return 'init_pred'
                else:
                    return 'reverse_pred'
        else:
            if x['reverse_pred'] not in conf.relation_type[reverse_task_type]:
                # init_pred and reverse_pred both are uncorrect relations
                return 'uncorrect'
            else:
                # reverse_pred is a correct relation but init_pred not
                return 'reverse_pred'
    label_df,tokenizer,model = Bert_model(args.task_type,'/kaggle/input/covid19-knowledge-graph/%s/'%args.task_type)
    marked_sentences,predict_data_loader = Prepare_predict_data(tokenizer,args.bs)
    model = model.to(device)
    preds = []
    preds_prob = []
    reverse_preds = []
    reverse_preds_prob = []
    for data in tqdm(predict_data_loader):
        ids,reverse_ids = [t.to(device) for t in data]
        outputs = model(input_ids=ids)
        logits = outputs[0]
        pred_prob, pred = torch.max(F.softmax(logits.data,1), 1)
        preds.extend(list(pred.cpu().detach().numpy()))
        preds_prob.extend(list(pred_prob.cpu().detach().numpy()))
        reverse_outputs = model(input_ids=reverse_ids)
        reverse_logits = reverse_outputs[0]
        reverse_pred_prob, reverse_pred = torch.max(F.softmax(reverse_logits.data,1), 1)
        reverse_preds.extend(list(reverse_pred.cpu().detach().numpy()))
        reverse_preds_prob.extend(list(reverse_pred_prob.cpu().detach().numpy()))

    pred_df = pd.DataFrame({'marked_sentence':marked_sentences,'init_pred':preds,'init_pred_prob':preds_prob,'reverse_pred':reverse_preds,'reverse_pred_prob':reverse_preds_prob})
    # map label(0, 1, 2...) to raw label(T, C, Sa...) 
    pred_df['init_pred'] = pred_df['init_pred'].replace(dict(label_df.set_index(['label'])['label_raw']))
    pred_df['reverse_pred'] = pred_df['reverse_pred'].replace(dict(label_df.set_index(['label'])['label_raw']))
    # judge the order of a pair of entities
    pred_df['filter'] = pred_df.apply(lambda x:Filter(x), axis=1)
    pred_df['pred'] = pred_df['init_pred']
    pred_df['pred_prob'] = pred_df['init_pred_prob']
    pred_df.loc[pred_df['filter']=='reverse_pred','pred'] = pred_df.loc[pred_df['filter']=='reverse_pred','reverse_pred']
    pred_df.loc[pred_df['filter']=='reverse_pred','pred_prob'] = pred_df.loc[pred_df['filter']=='reverse_pred','reverse_pred_prob']
    pred_df = pred_df.loc[pred_df['filter']!='uncorrect']
    pred_df = marked_sentence_df.merge(pred_df,how='inner',on='marked_sentence')
    pred_df['init_start_entity'] = pred_df['start_entity']
    pred_df['init_start_entity_type'] = pred_df['start_entity_type']
    pred_df.loc[pred_df['filter']=='reverse_pred','start_entity'] = pred_df.loc[pred_df['filter']=='reverse_pred','end_entity']
    pred_df.loc[pred_df['filter']=='reverse_pred','start_entity_type'] = pred_df.loc[pred_df['filter']=='reverse_pred','end_entity_type']
    pred_df.loc[pred_df['filter']=='reverse_pred','end_entity'] = pred_df.loc[pred_df['filter']=='reverse_pred','init_start_entity']
    pred_df.loc[pred_df['filter']=='reverse_pred','end_entity_type'] = pred_df.loc[pred_df['filter']=='reverse_pred','init_start_entity_type']
    pred_df.drop(['init_start_entity','init_start_entity_type'],axis=1,inplace=True)
    torch.cuda.empty_cache()
    return label_df,pred_df

In [9]:
# chemical-disease relation prediction
args.task_type = 'chemical-disease'
c_d_label_df,c_d_pred_df = Predict()

In [10]:
# chemical-disease relation theme
c_d_label_df

,label_raw,label,theme
0,C,0,Inhibits cell growth (esp. cancers)
1,J,1,Role in pathogenesis
2,Mp,2,Biomarkers (progression)
3,Pa,3,"Alleviates, reduces"
4,Pr,4,"Prevents, suppresses"
5,Sa,5,Side effect/adverse event
6,T,6,Treatment/therapy (incl. investigatory)


In [11]:
# chemical-disease classification results
c_d_pred_df.sample(5)

,start_entity,end_entity,start_entity_type,end_entity_type,marked_sentence,init_pred,init_pred_prob,reverse_pred,reverse_pred_prob,filter,pred,pred_prob
725,Glycine,Respiratory Tract Infections,Chemical,Disease,"Ehlayel, Abdul Bari Bener A607 Strong Associat...",T,0.580941,T,0.580941,init_pred,T,0.580941
321,Ginseng,Cough,Chemical,Disease,Pieters A245 How to Define Chronic end_entity:...,J,0.997718,J,0.999964,init_pred,J,0.997718
423,Asthma,Montelukast,Disease,Chemical,1 Million South Korean Health-Care Records Gwa...,Mp,0.992673,Mp,0.992673,reverse_pred,Mp,0.992673
431,Lung Neoplasms,Montelukast,Disease,Chemical,1 Million South Korean Health-Care Records Gwa...,Mp,0.992673,Mp,0.992673,reverse_pred,Mp,0.992673
756,Glatiramer Acetate,Egg Hypersensitivity,Chemical,Disease,"Ehlayel, Abdul Bari Bener A607 Strong Associat...",T,0.580941,T,0.580941,init_pred,T,0.580941


In [12]:
# chemical-gene relation prediction
args.task_type = 'chemical-gene'
c_g_label_df,c_g_pred_df = Predict()

In [13]:
# chemical-gene relation theme
c_g_label_df

,label_raw,label,theme
0,A+,0,"Agonism, activation"
1,A-,1,"Antagonism, blocking"
2,B,2,"Binding, ligand (esp. receptors)"
3,E,3,Affects expression/production (neutral)
4,E-,4,Decreases expression/production
5,K,5,"Metabolism, pharmacokinetics"
6,N,6,Inhibits
7,O,7,"Transport, channels"
8,Z,8,Enzyme activity


In [14]:
# chemical-gene classification results
c_g_pred_df.sample(5)

,start_entity,end_entity,start_entity_type,end_entity_type,marked_sentence,init_pred,init_pred_prob,reverse_pred,reverse_pred_prob,filter,pred,pred_prob
315,Chloroquine,NP,Chemical,Gene,start_entity diphosphate CQ is a hydrophilic d...,E,0.968937,K,0.615419,init_pred,E,0.968937
172,Toluene,EIF1,Chemical,Gene,"Leung, Ka Hou Chu A67 The relationship between...",B,0.729221,B,0.729221,init_pred,B,0.729221
162,Doxycycline,PTX3,Chemical,Gene,"Leung, Ka Hou Chu A67 The relationship between...",E,0.998353,O,0.968128,init_pred,E,0.998353
399,Imiquimod,CXCL8,Chemical,Gene,start_entity also induced the expression of in...,E-,0.656016,E,0.989915,init_pred,E-,0.656016
415,PLpro,GRL0617,Gene,Chemical,From conventional molecular dynamics and accel...,K,0.625306,K,0.920474,reverse_pred,K,0.920474


In [15]:
# gene-disease relation prediction
args.task_type = 'gene-disease'
g_d_label_df,g_d_pred_df = Predict()

In [16]:
# gene-disease relation theme
g_d_label_df

,label_raw,label,theme
0,D,0,Drug targets
1,G,1,Promotes progression
2,J,2,Role in pathogenesis
3,L,3,Improper regulation linked to disease
4,Md,4,Biomarkers (diagnostic)
5,Te,5,Possible therapeutic effect
6,U,6,Causal mutations
7,Ud,7,Mutations affect disease course
8,X,8,Overexpression in disease
9,Y,9,Polymorphisms alter risk


In [17]:
# gene-disease classification results
g_d_pred_df.sample(5)

,start_entity,end_entity,start_entity_type,end_entity_type,marked_sentence,init_pred,init_pred_prob,reverse_pred,reverse_pred_prob,filter,pred,pred_prob
3262,Drug Eruptions,VDR,Disease,Gene,Pieters A245 How to Define Chronic Cough: Base...,L,0.998743,L,0.998743,reverse_pred,L,0.998743
6950,CSF3,Melanoma,Gene,Disease,"nnAbbreviationsAHR, airway hyperresponsiveness...",J,0.999616,L,0.999203,init_pred,J,0.999616
7697,ERAP2,Feline Infectious Peritonitis,Gene,Disease,"Five candidate genes ELMO1, RRAGA, TNFSF10, ER...",U,0.864559,U,0.997139,init_pred,U,0.864559
6539,ERVW-1,COVID-19,Gene,Disease,The severe acute respiratory syndrome end_enti...,Y,0.524023,Y,0.542540,init_pred,Y,0.524023
4490,SAA1,diarrhea,Gene,Disease,The results indicated that increased Hp concen...,J,0.999970,J,0.999569,init_pred,J,0.999970


In [18]:
# gene-gene relation prediction
args.task_type = 'gene-gene'
g_g_label_df,g_g_pred_df = Predict()

In [19]:
# gene-gene relation theme
g_g_label_df

,label_raw,label,theme
0,B,0,"Binding, ligand (esp. receptors)"
1,E,1,Affects expression/production (neutral)
2,E+,2,Increases expression/production
3,H,3,Same protein or complex
4,I,4,Signaling pathway
5,Q,5,Production by cell population
6,Rg,6,Regulation
7,V+,7,"Activates, stimulates"
8,W,8,Enhances response


In [20]:
# gene-gene classification results
g_g_pred_df.sample(5)

,start_entity,end_entity,start_entity_type,end_entity_type,marked_sentence,init_pred,init_pred_prob,reverse_pred,reverse_pred_prob,filter,pred,pred_prob
11664,CSF1,CSF3,Gene,Gene,Emodin significantly inhibited the production ...,E+,0.987432,E+,0.978247,init_pred,E+,0.987432
13625,GCNT2,REN,Gene,Gene,Reverse transcription coupled to gene-specific...,E+,0.654601,E+,0.552663,init_pred,E+,0.654601
6737,IL4,TNF,Gene,Gene,OBJECTIVE: To investigate an association of va...,E+,0.958611,E+,0.914068,init_pred,E+,0.958611
13482,IFIH1,IRF1,Gene,Gene,Analysis of the 200 to 300 regulated genes sho...,V+,0.432530,I,0.376433,init_pred,V+,0.432530
3441,TICAM1,CXCL1,Gene,Gene,"Furthermore, both Escherichia coli and Pseudom...",Rg,0.818656,Q,0.878697,reverse_pred,Q,0.878697


# Show relation between covid-19 and other entities

pycharts cannot show multiple relations between two entities, so the graph is incomplete. Complete relations can be seen in ***pred_df***.

In [21]:
# chemicl-COVID-19 relations
g = Build_graph(c_d_pred_df.loc[(c_d_pred_df['start_entity']=='COVID-19')|(c_d_pred_df['end_entity']=='COVID-19')],relation=True,repulsion=800,title='chemical-COVID-19 knowledge graph',labelShow=True)
g

In [22]:
# gene-COVID-19 relations
g = Build_graph(g_d_pred_df.loc[(g_d_pred_df['start_entity']=='COVID-19')|(g_d_pred_df['end_entity']=='COVID-19')],relation=True,repulsion=60,title='gene-COVID-19 knowledge graph',labelShow=False)
g

In [23]:
# gene-COVID-19 relations
g = Build_graph(marked_sentence_df.loc[(marked_sentence_df['start_entity']=='COVID-19')&(marked_sentence_df['end_entity_type']=='Disease')|(marked_sentence_df['start_entity_type']=='Disease')&(marked_sentence_df['end_entity']=='COVID-19')],relation=False,repulsion=60,title='disease-COVID-19 topology graph',labelShow=False)
g

# Results and answer for task

In [24]:
# merge all relation prediction and save results
cols = ['start_entity','end_entity','start_entity_type','end_entity_type','marked_sentence','pred','pred_prob']
relation_df = pd.concat([c_d_pred_df[cols],c_g_pred_df[cols],g_d_pred_df[cols],g_g_pred_df[cols]]).append(marked_sentence_df.loc[(marked_sentence_df['start_entity_type'].isin(['Chemical','Disease'])&(marked_sentence_df['start_entity_type']==marked_sentence_df['end_entity_type']))]).reset_index(drop=True)
relation_df.loc[(relation_df['pred'].isna())&(relation_df['start_entity_type']=='Chemical'),'pred'] = 'CC'
relation_df.loc[(relation_df['pred'].isna())&(relation_df['start_entity_type']=='Disease'),'pred'] = 'DD'
relation_df = relation_df[cols]
relation_df.to_csv('relation.csv',index=False)

In [25]:
relation_df

,start_entity,end_entity,start_entity_type,end_entity_type,marked_sentence,pred,pred_prob
0,keyhole limpet hemocyanin,"Lupus Erythematosus, Systemic",Chemical,Disease,"Middle-aged female identical twins, one of who...",Pa,0.969853
1,Cocaine,Incontinentia Pigmenti,Chemical,Disease,"start_entity 10 mgkg, administered intraperito...",Pa,0.832485
2,Daptomycin,COVID-19,Chemical,Disease,"start_entity is not indicated for end_entity, ...",T,0.999757
3,Zinc,diarrhea,Chemical,Disease,start_entity is being used for the treatment o...,Pa,0.769986
4,Zinc,Glycogen Storage Disease Type II,Chemical,Disease,start_entity was effective in decreasing the i...,Pa,0.423429
...,...,...,...,...,...,...,...
39577,Coronavirus Infections,Severe Acute Respiratory Syndrome,Disease,Disease,In a recent review on the persistence of human...,DD,NaN
39578,Coronavirus Infections,Syndrome,Disease,Disease,In a recent review on the persistence of human...,DD,NaN
39579,MERS,Severe Acute Respiratory Syndrome,Disease,Disease,In a recent review on the persistence of human...,DD,NaN
39580,MERS,Syndrome,Disease,Disease,In a recent review on the persistence of human...,DD,NaN


In [26]:
# subsample of knowledge graph
g = Build_graph(relation_df.sample(1000),relation=True,repulsion=15,title='subsample of COVID-19 knowledge graph',labelShow=False)
g

## What do we know about COVID-19 risk factors?

In [27]:
# 1. What chemicals have Sa(Side effect/adverse event) to COVID-19 and revelant disease?

# due to a large number of negative words, these predictions may be wrong
c_d_pred_df.loc[c_d_pred_df['pred']=='Sa']

,start_entity,end_entity,start_entity_type,end_entity_type,marked_sentence,init_pred,init_pred_prob,reverse_pred,reverse_pred_prob,filter,pred,pred_prob
5,ventilation,"Respiratory Distress Syndrome, Adult",Chemical,Disease,Hospital-acquired end_entity may result from a...,Sa,0.954768,Sa,0.997769,init_pred,Sa,0.954768
7,Nelfinavir,Severe Acute Respiratory Syndrome,Chemical,Disease,start_entity inhibited the cytopathic effect i...,Sa,0.559937,C,0.601428,init_pred,Sa,0.559937
37,ventilation,"Corneal Dystrophy, Fleck",Chemical,Disease,Findings end_entity simulation revealed that t...,Sa,0.747980,Pr,0.768366,init_pred,Sa,0.747980
42,ventilation,COVID-19,Chemical,Disease,The main complications reported in hospitalize...,Sa,0.976008,Sa,0.957271,init_pred,Sa,0.976008
44,glucocorticoids,Lymphopenia,Chemical,Disease,"end_entity can be caused by start_entity, and ...",Sa,0.981520,Pr,0.482003,init_pred,Sa,0.981520
...,...,...,...,...,...,...,...,...,...,...,...,...
862,ventilation,COVID-19,Chemical,Disease,Severe_Acute_Respiratory_Syndrome end_entity d...,Sa,0.748146,C,0.995705,init_pred,Sa,0.748146
864,mesenchymal stem cells,BMD,Chemical,Disease,The injection of Ci-I hydrogels encapsulating ...,Sa,0.765677,T,0.818385,init_pred,Sa,0.765677
880,Hexachlorophene,inhibitory concentration 50,Chemical,Disease,"start_entity end_entity: 6M, tannic acid end_e...",Sa,0.816924,Pa,0.654464,init_pred,Sa,0.816924
882,ventilation,"Respiratory Distress Syndrome, Adult",Chemical,Disease,"Compared with survivors, non-survivors were ol...",Sa,0.824087,J,0.619186,init_pred,Sa,0.824087


In [28]:
# 2. What genes have U(Causal mutations), Ud(Mutations affect disease course), Y(Polymorphisms alter risk) to COVID-19?


g_d_pred_df.loc[(g_d_pred_df['end_entity']=='COVID-19')&(g_d_pred_df['pred'].isin(['U','Ud','Y']))]

,start_entity,end_entity,start_entity_type,end_entity_type,marked_sentence,init_pred,init_pred_prob,reverse_pred,reverse_pred_prob,filter,pred,pred_prob
1,p28,COVID-19,Gene,Disease,A high level of similarity was detected betwee...,Ud,0.932619,U,0.975709,init_pred,Ud,0.932619
4,E1,COVID-19,Gene,Disease,The carbohydrate contents of end_entity glycop...,U,0.959630,U,0.977801,init_pred,U,0.959630
5,E2,COVID-19,Gene,Disease,The carbohydrate contents of end_entity glycop...,U,0.935260,J,0.610730,init_pred,U,0.935260
9,ERVW-1,COVID-19,Gene,Disease,Tunicamycin has different effects on the glyco...,Ud,0.687815,Y,0.812220,init_pred,Ud,0.687815
11,E1,COVID-19,Gene,Disease,The end_entity envelope ERVW-1 start_entity ap...,Ud,0.989257,L,0.718455,init_pred,Ud,0.989257
...,...,...,...,...,...,...,...,...,...,...,...,...
7807,S2,COVID-19,Gene,Disease,The severe acute respiratory syndrome end_enti...,Y,0.676709,U,0.480307,init_pred,Y,0.676709
7819,ACE2,COVID-19,Gene,Disease,Molecular modeling revealed that end_entity RB...,U,0.607001,Y,0.998896,init_pred,U,0.607001
7835,ACE2,COVID-19,Gene,Disease,Infection by severe acute respiratory syndrome...,U,0.928479,U,0.836396,init_pred,U,0.928479
7862,MERTK,COVID-19,Gene,Disease,A 12 start_entity peptide derived from end_ent...,Ud,0.558342,J,0.512183,init_pred,Ud,0.558342


In [29]:
# 3. What diseases are associated to COVID-19 (like complication)?
d_d_df = relation_df.loc[((relation_df['start_entity']=='COVID-19')&(relation_df['end_entity_type']=='Disease'))|((relation_df['start_entity_type']=='Disease')&(relation_df['end_entity']=='COVID-19'))]
value_counts_dic = dict(d_d_df['start_entity'].value_counts())
end_entity_value_counts_dic = dict(d_d_df['end_entity'].value_counts())
for key in end_entity_value_counts_dic:
    if key in value_counts_dic:
        value_counts_dic[key] += end_entity_value_counts_dic[key]
    else:
        value_counts_dic[key] = end_entity_value_counts_dic[key]
most_relevant_disease = []
for key in value_counts_dic:
    if value_counts_dic[key] > 10:
        most_relevant_disease.append(key)
relation_df.loc[((relation_df['start_entity'].isin(most_relevant_disease))&(relation_df['end_entity']=='COVID-19'))|((relation_df['start_entity']=='COVID-19')&(relation_df['end_entity'].isin(most_relevant_disease)))]

,start_entity,end_entity,start_entity_type,end_entity_type,marked_sentence,pred,pred_prob
30951,COVID-19,Cataract congenital Volkmann type,Disease,Disease,The pathogenesis of canine start_entity end_en...,DD,NaN
30956,COVID-19,Cataract congenital Volkmann type,Disease,Disease,The genomic organization at the 3 end of canin...,DD,NaN
30961,COVID-19,"Influenza, Human",Disease,Disease,Blood was taken from the sows at the beginning...,DD,NaN
30964,COVID-19,Cataract congenital Volkmann type,Disease,Disease,From the reasons that canine start_entity end_...,DD,NaN
30971,COVID-19,Infection,Disease,Disease,end_entity of cell monolayers by murine start_...,DD,NaN
...,...,...,...,...,...,...,...
39569,COVID-19,Severe Acute Respiratory Syndrome,Disease,Disease,end_entity start_entity end_entity-CoV papain-...,DD,NaN
39572,COVID-19,Coronavirus Infections,Disease,Disease,In a recent review on the persistence of human...,DD,NaN
39573,COVID-19,MERS,Disease,Disease,In a recent review on the persistence of human...,DD,NaN
39574,COVID-19,Severe Acute Respiratory Syndrome,Disease,Disease,In a recent review on the persistence of human...,DD,NaN


## What do we know about vaccines and therapeutics?

In [30]:
# 1. What chemicals have Pa(Alleviates, reduces), Pr(Prevents, suppresses), T(Treatment/therapy (incl. investigatory)) to COVID-19 and revelant disease?

c_d_pred_df.loc[(c_d_pred_df['end_entity']=='COVID-19')&(c_d_pred_df['pred'].isin(['Pa','Pr','T']))]

,start_entity,end_entity,start_entity_type,end_entity_type,marked_sentence,init_pred,init_pred_prob,reverse_pred,reverse_pred_prob,filter,pred,pred_prob
2,Daptomycin,COVID-19,Chemical,Disease,"start_entity is not indicated for end_entity, ...",T,0.999757,T,0.597799,init_pred,T,0.999757
15,ventilation,COVID-19,Chemical,Disease,Study objectives: To report on our experience ...,T,0.999976,Sa,0.998793,init_pred,T,0.999976
47,oseltamivir,COVID-19,Chemical,Disease,During the pandemic H1N1 2009 influenza A viru...,T,0.584094,T,0.932823,init_pred,T,0.584094
55,ventilation,COVID-19,Chemical,Disease,"Bad housing, particularly bad start_entity, wi...",Pr,0.759886,Sa,0.961628,init_pred,Pr,0.759886
57,Ganciclovir,COVID-19,Chemical,Disease,start_entity can be considered for patients wi...,T,0.999927,T,0.999898,init_pred,T,0.999927
59,ventilation,COVID-19,Chemical,Disease,Case report We report a case of acute respirat...,T,0.848494,Sa,0.999983,init_pred,T,0.848494
67,Chloroquine,COVID-19,Chemical,Disease,"start_entity, remdesivir, lopinavir, ribavirin...",T,0.948416,Pr,0.833724,init_pred,T,0.948416
69,ventilation,COVID-19,Chemical,Disease,Three clinical cases of patients presenting se...,T,0.946440,T,0.936319,init_pred,T,0.946440
78,ventilation,COVID-19,Chemical,Disease,The populations of patients with community-acq...,T,0.999792,C,0.627967,init_pred,T,0.999792
79,ventilation,COVID-19,Chemical,Disease,"By contrast, the use of non-invasive start_ent...",T,0.998000,T,0.521081,init_pred,T,0.998000


In [31]:
# 2. What genes have D(Drug targets), G(Promotes progression), J(Role in pathogenesis), Te(Possible therapeutic effect), X(Overexpression in disease) to COVID-19 and revelant disease?

g_d_pred_df.loc[(g_d_pred_df['end_entity']=='COVID-19')&(g_d_pred_df['pred'].isin(['D','G','J','Te','X']))]

,start_entity,end_entity,start_entity_type,end_entity_type,marked_sentence,init_pred,init_pred_prob,reverse_pred,reverse_pred_prob,filter,pred,pred_prob
8,GCNT2,COVID-19,Gene,Disease,"This end_entity, like influenza virus, can rep...",J,0.999964,Ud,0.578843,init_pred,J,0.999964
18,PSMD1,COVID-19,Gene,Disease,The sequences of the peplomeric start_entity p...,J,0.958232,U,0.994972,init_pred,J,0.958232
24,CYTIP,COVID-19,Gene,Disease,"The hemagglutininesterase start_entity, spike ...",D,0.480010,Te,0.834432,init_pred,D,0.480010
27,S2,COVID-19,Gene,Disease,These observations may reflect the biological ...,Te,0.731474,J,0.999688,init_pred,Te,0.731474
33,CYTIP,COVID-19,Gene,Disease,"In contrast to bovine end_entity, which appare...",J,0.993721,L,0.988948,init_pred,J,0.993721
...,...,...,...,...,...,...,...,...,...,...,...,...
7805,PSMD1,COVID-19,Gene,Disease,We investigated the kinetics of serologic resp...,Te,0.744864,J,0.997138,init_pred,Te,0.744864
7808,NDUFB6,COVID-19,Gene,Disease,We fitted a log-normal distribution to pooled ...,D,0.962256,Te,0.909174,init_pred,D,0.962256
7810,IFNA1,COVID-19,Gene,Disease,Poxvirus and end_entity mutants that lacked 2-...,Te,0.784429,J,0.973894,init_pred,Te,0.784429
7834,ARIH1,COVID-19,Gene,Disease,The Human respiratory syncytial virus RSV is o...,Te,0.914881,J,0.916928,init_pred,Te,0.914881


# Future work

The classification model is just a baseline. We will enrich medical vocabulary and do further fune-tuning. On the other hand, reasoning unknown relations is an important application of knowledge graph. We are trying to find candidate drugs or other treatments for COVID-19 via graph embedding techniques.

# Acknowledgement

This work is supported by [Aladdin Healthcare Technologies](https://aladdinid.com/)
<span style="display: inline-block;"> <img src="https://github.com/jxzly/Biomedical-Relation-Classification/blob/master/figure/Aladdin.png?raw=true" width=40/></span>